In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
df = spark \
        .readStream \
        .format("text") \
        .option("cleanSource", "archive") \
        .option("sourceArchiveDir", "/home/jovyan/archive") \
        .load("/home/jovyan/notebooks/data/strucutred_streaming_files/")

In [ ]:
query = df \
        .writeStream \
        .outputMode("append") \
        .trigger(processingTime='20 seconds') \
        .format("console") \
        .start()

query.awaitTermination()

In [ ]:
query.stop()

In [ ]:
sc.stop()